In [ ]:
from autumn.infrastructure.remote import springboard
from autumn.projects.sm_covid2.common_school.runner_tools import (
    run_full_analysis, 
    print_continuous_status, 
    download_analysis,
    DEFAULT_RUN_CONFIG,
    TEST_RUN_CONFIG,
)

### Define task function

#### Standard config

In [ ]:
DEFAULT_RUN_CONFIG

In [ ]:
run_config = DEFAULT_RUN_CONFIG  # update if required

In [ ]:
ISO3 = "FRA"
ANALYSIS = "main"

In [ ]:
def remote_full_analysis_task(bridge: springboard.task.TaskBridge, iso3: str = 'FRA', analysis: str = "main"):
    
    import multiprocessing as mp
    mp.set_start_method('forkserver')

    bridge.logger.info(f"Running full analysis for {iso3}. Analysis type: {analysis}.")

    idata, uncertainty_df, diff_quantiles_df = run_full_analysis(
        iso3,
        analysis=analysis, 
        run_config=run_config,
        output_folder=bridge.out_path,
        logger=bridge.logger
    )
   
    bridge.logger.info("Full analysis complete")

In [ ]:
mspec = springboard.EC2MachineSpec(run_config['n_cores'], 4, "compute")
task_kwargs = {
    "iso3": ISO3,
    "analysis": ANALYSIS
}
tspec = springboard.TaskSpec(remote_full_analysis_task, task_kwargs)

In [ ]:
analysis_title = "test_4cpu"
config_str = f"_{ANALYSIS}_LHS{run_config['n_opti_searches']}_opt{run_config['opti_budget']}_mc{run_config['metropolis_tune']}n{run_config['metropolis_draws']}"

run_path = springboard.launch.get_autumn_project_run_path("school_project", ISO3, analysis_title + config_str)
run_path

In [ ]:
runner = springboard.launch.launch_synced_autumn_task(tspec, mspec, run_path)

In [ ]:
runner.s3.get_status()

In [ ]:
print(runner.top("%MEM"))

In [ ]:
# wait function with status printing
# print_continuous_status(runner)

In [ ]:
# run_path = 'projects/school_project/FRA/2023-08-22T1004-single_start_main_LHS4_opt10000_mc2000n10000'

In [ ]:
download_analysis(run_path)